In [150]:
import pandas as pd
import numpy as np
from src.api_functions import *
from src.cleaning_functions import *

In [151]:
# df1 = dataframe from API
# df2 = dataframe from energy .csv
# df3 = dataframe from merging df1 and a modified version of df2
# df4 = dataframe derived from df3 containing only the year 2020
# df5 = dataframe derived from df4 without NaNs and with grouped columns by categories
# df6 = dataframe from avg-rate .csv (population)

In [152]:
csv1 = 'data/population-data.csv'
csv2 = 'data/energy-data/owid-energy-data.csv'
csv3 = 'data/avg-rate.csv'

In [153]:
df1 = load_csv(csv1)
df2 = load_csv(csv2)
df6 = load_csv(csv3)

In [154]:
df1.shape

(235, 4)

In [155]:
df1.head()

,country_name,population,ranking,world_share
0,China,1439323776,1,18.464154
1,India,1380004385,2,17.703184
2,United States,331002651,3,4.246219
3,Indonesia,273523615,4,3.508858
4,Pakistan,220892340,5,2.833685


In [156]:
df2.shape

(17432, 122)

In [157]:
df2.head()

,iso_code,country,year,coal_prod_change_pct,coal_prod_change_twh,gas_prod_change_pct,gas_prod_change_twh,oil_prod_change_pct,oil_prod_change_twh,energy_cons_change_pct,...,solar_elec_per_capita,solar_energy_per_capita,gdp,wind_share_elec,wind_cons_change_pct,wind_share_energy,wind_cons_change_twh,wind_consumption,wind_elec_per_capita,wind_energy_per_capita
0,AFG,Afghanistan,1900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AFG,Afghanistan,1901,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AFG,Afghanistan,1902,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AFG,Afghanistan,1903,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AFG,Afghanistan,1904,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [158]:
df6.shape

(14940, 25)

In [159]:
df6.head()

,LocID,Location,VarID,Variant,Time,MidPeriod,TFR,NRR,CBR,Births,...,CDR,Deaths,DeathsMale,DeathsFemale,CNMR,NetMigrations,GrowthRate,NatIncr,SRB,MAC
0,4,Afghanistan,2,Medium,1950-1955,1953,7.45,1.636,50.314,2015.476,...,36.862,1476.605,794.020,682.585,-0.499,-20.0,1.296,13.452,1.06,29.835
1,4,Afghanistan,2,Medium,1955-1960,1958,7.45,1.765,50.998,2201.589,...,33.718,1455.605,783.110,672.495,-0.463,-20.0,1.683,17.280,1.06,29.835
2,4,Afghanistan,2,Medium,1960-1965,1963,7.45,1.899,51.477,2439.130,...,30.808,1459.784,777.437,682.347,-0.422,-20.0,2.026,20.669,1.06,29.835
3,4,Afghanistan,2,Medium,1965-1970,1968,7.45,2.017,51.646,2728.221,...,28.223,1490.896,790.745,700.151,-0.379,-20.0,2.307,23.423,1.06,29.835
4,4,Afghanistan,2,Medium,1970-1975,1973,7.45,2.141,51.234,3056.451,...,25.495,1520.938,802.414,718.524,-0.335,-20.0,2.544,25.739,1.06,29.835


In [160]:
df6.columns

Index(['LocID', 'Location', 'VarID', 'Variant', 'Time', 'MidPeriod', 'TFR',
       'NRR', 'CBR', 'Births', 'LEx', 'LExMale', 'LExFemale', 'IMR', 'Q5',
       'CDR', 'Deaths', 'DeathsMale', 'DeathsFemale', 'CNMR', 'NetMigrations',
       'GrowthRate', 'NatIncr', 'SRB', 'MAC'],
      dtype='object')

In [161]:
df6.drop(['LocID', 'VarID', 'Variant', 'Time', 'TFR',
       'NRR', 'CBR', 'Births', 'LEx', 'LExMale', 'LExFemale', 'IMR', 'Q5',
       'CDR', 'Deaths', 'DeathsMale', 'DeathsFemale', 'CNMR', 'NetMigrations',
          'NatIncr', 'SRB', 'MAC'], axis=1, inplace=True)

In [162]:
df6.head() # We keep this value for an ulterior representation

,Location,MidPeriod,GrowthRate
0,Afghanistan,1953,1.296
1,Afghanistan,1958,1.683
2,Afghanistan,1963,2.026
3,Afghanistan,1968,2.307
4,Afghanistan,1973,2.544


In [163]:
df6.set_index('MidPeriod',drop=True,inplace=True)

In [164]:
df7 = df6.groupby(['Location'],as_index = False).mean() # For the growth prediction

In [165]:
df7.dropna(axis=0, how='any')

,Location,GrowthRate
0,Afghanistan,1.512467
1,Africa,1.955567
2,African Group,1.955967
3,African Union,1.956133
4,African Union: Central Africa,2.149567
...,...,...
489,World,0.970467
490,World Bank Regional Groups (developing only),1.098833
492,Yemen,1.622800
493,Zambia,2.375733


In [166]:
df0 = df1

In [167]:
df0.columns

Index(['country_name', 'population', 'ranking', 'world_share'], dtype='object')

In [168]:
list_a = df0['country_name'] # df1 --> population
list_b = df7['Location'] # df2 --> energy
exclusion_list_2 = np.setdiff1d(list_b,list_a)
df7_mod = df7[~df7['Location'].isin(exclusion_list_2)]
df0.rename(columns={"country_name": "country"}, inplace=True)
df7_mod.rename(columns={"Location": "country"}, inplace=True)
df0.set_index('country', inplace=True)
df7_mod.set_index('country', inplace=True)
df_avg_rate = df0.merge(df7_mod, how='inner', on='country') # We combine both datasets through the common countries. The
# uncommon countries impact is estimated apart

/home/krujo/Anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [169]:
df_avg_rate.shape

(204, 4)

In [174]:
df_avg_rate = df_avg_rate.groupby(['country'],as_index = True).mean()

In [175]:
df_avg_rate.shape

(204, 4)

In [172]:
df_avg_rate.head()

,population,ranking,world_share,GrowthRate
country,,,,
Afghanistan,38928346,37,0.499387,1.512467
Albania,2877797,140,0.036917,-0.099333
Algeria,43851044,33,0.562537,1.383767
American Samoa,55191,210,0.000708,0.431033
Andorra,77265,203,0.000991,1.539667


In [140]:
df_avg_rate.to_csv('output/pop_avg_rate.csv' , index = True)

In [142]:
df7['GrowthRate'].mean()

1.0011267229254572

In [19]:
len(df1['country_name'].unique())

235

In [20]:
list_1 = df1['country_name']

In [21]:
len(df2['country'].unique())

242

In [22]:
list_2 = df2['country']

In [23]:
# Let's see which countries from list 2 that ARE NOT in list 1. List 1 is the most relevant, so let's remove list 2 elements
# that are not present
# list 1 --> population
# list 2 --> energy
exclusion_list = np.setdiff1d(list_2,list_1)

In [24]:
print(exclusion_list)

['Africa' 'Asia Pacific' 'Brunei' 'CIS' 'Cape Verde' 'Central America'
 "Cote d'Ivoire" 'Czechia' 'Czechoslovakia' 'Democratic Republic of Congo'
 'Eastern Africa' 'Europe' 'Europe (other)' 'Falkland Islands ' 'Kosovo'
 'Macau' 'Middle Africa' 'Middle East' 'Netherlands Antilles'
 'North America' 'OPEC' 'Other Asia & Pacific' 'Other CIS'
 'Other Caribbean' 'Other Middle East' 'Other Northern Africa'
 'Other South America' 'Other Southern Africa' 'Palestine' 'Reunion'
 'Saint Kitts and Nevis' 'Saint Pierre and Miquelon'
 'Saint Vincent and the Grenadines' 'Sao Tome and Principe'
 'South & Central America' 'Timor' 'Turks and Caicos Islands' 'USSR'
 'United States Pacific Islands' 'United States Virgin Islands'
 'Western Africa' 'World' 'Yugoslavia']


In [25]:
# df_ec = dataframe 2, exclusived countries
df_ec = df2[df2['country'].isin(exclusion_list)]

In [26]:
df_ec.shape

(2750, 122)

In [27]:
df2_mod = df2[~df2['country'].isin(exclusion_list)]

In [28]:
df2_mod = df2_mod

In [29]:
df2_mod.shape

(14682, 122)

In [30]:
df1.rename(columns={"country_name": "country"}, inplace=True)

In [31]:
df1.set_index('country', inplace=True)
df2_mod.set_index('country', inplace=True)

In [32]:
df3 = df1.merge(df2_mod, how='inner', on='country')

In [33]:
# I can use df3 for having a start point for the prediction at 2020
df3.head()

,population_x,ranking,world_share,iso_code,year,coal_prod_change_pct,coal_prod_change_twh,gas_prod_change_pct,gas_prod_change_twh,oil_prod_change_pct,...,solar_elec_per_capita,solar_energy_per_capita,gdp,wind_share_elec,wind_cons_change_pct,wind_share_energy,wind_cons_change_twh,wind_consumption,wind_elec_per_capita,wind_energy_per_capita
country,,,,,,,,,,,,,,,,,,,,,
China,1439323776,1,18.464154,CHN,1900,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,3.360000e+11,NaN,NaN,NaN,NaN,NaN,NaN,NaN
China,1439323776,1,18.464154,CHN,1901,NaN,0.000,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
China,1439323776,1,18.464154,CHN,1902,NaN,0.291,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
China,1439323776,1,18.464154,CHN,1903,1972.001,5.731,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
China,1439323776,1,18.464154,CHN,1904,6.467,0.389,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
# df4 will be the starting point
df4 = df3[df3['year'] == 2020]

In [35]:
df4.head()

,population_x,ranking,world_share,iso_code,year,coal_prod_change_pct,coal_prod_change_twh,gas_prod_change_pct,gas_prod_change_twh,oil_prod_change_pct,...,solar_elec_per_capita,solar_energy_per_capita,gdp,wind_share_elec,wind_cons_change_pct,wind_share_energy,wind_cons_change_twh,wind_consumption,wind_elec_per_capita,wind_energy_per_capita
country,,,,,,,,,,,,,,,,,,,,,
China,1439323776,1,18.464154,CHN,2020,NaN,NaN,NaN,NaN,NaN,...,181.405,NaN,NaN,6.119,NaN,NaN,NaN,NaN,324.111,NaN
India,1380004385,2,17.703184,IND,2020,NaN,NaN,NaN,NaN,NaN,...,42.557,NaN,NaN,4.502,NaN,NaN,NaN,NaN,43.779,NaN
United States,331002651,3,4.246219,USA,2020,NaN,NaN,NaN,NaN,NaN,...,400.693,NaN,NaN,8.310,NaN,NaN,NaN,NaN,1016.709,NaN
Pakistan,220892340,5,2.833685,PAK,2020,NaN,NaN,NaN,NaN,NaN,...,4.664,NaN,NaN,2.194,NaN,NaN,NaN,NaN,14.444,NaN
Brazil,212559417,6,2.726787,BRA,2020,NaN,NaN,NaN,NaN,NaN,...,35.723,NaN,NaN,9.357,NaN,NaN,NaN,NaN,266.644,NaN


In [36]:
df4.shape

(67, 124)

In [37]:
df4.isna().sum()

population_x               0
ranking                    0
world_share                0
iso_code                   0
year                       0
                          ..
wind_share_energy         67
wind_cons_change_twh      67
wind_consumption          67
wind_elec_per_capita       0
wind_energy_per_capita    67
Length: 124, dtype: int64

In [38]:
df4.dropna(axis=1, how="all", inplace=True)

<ipython-input-38-b6a2d0dddd85>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df4.dropna(axis=1, how="all", inplace=True)


In [39]:
df4.shape

(67, 45)

In [40]:
df4.head()

,population_x,ranking,world_share,iso_code,year,biofuel_share_elec,biofuel_elec_per_capita,carbon_intensity_elec,coal_share_elec,coal_elec_per_capita,...,oil_elec_per_capita,other_renewables_elec_per_capita,other_renewables_share_elec,per_capita_electricity,renewables_elec_per_capita,renewables_share_elec,solar_share_elec,solar_elec_per_capita,wind_share_elec,wind_elec_per_capita
country,,,,,,,,,,,,,,,,,,,,,
China,1439323776,1,18.464154,CHN,2020,1.700,90.070,NaN,60.748,3217.632,...,111.384,90.139,1.702,5296.721,1537.208,29.022,3.425,181.405,6.119,324.111
India,1380004385,2,17.703184,IND,2020,1.071,10.415,NaN,70.563,686.181,...,0.096,11.293,1.161,972.437,216.116,22.224,4.376,42.557,4.502,43.779
United States,331002651,3,4.246219,USA,2020,1.384,169.348,NaN,19.107,2337.762,...,87.453,220.496,1.802,12235.088,2501.304,20.444,3.275,400.693,8.310,1016.709
Pakistan,220892340,5,2.833685,PAK,2020,1.840,12.113,NaN,20.157,132.732,...,156.246,12.113,1.840,658.476,245.336,37.258,0.708,4.664,2.194,14.444
Brazil,212559417,6,2.726787,BRA,2020,9.037,257.535,NaN,3.598,102.532,...,39.197,257.535,9.037,2849.682,2400.323,84.231,1.254,35.723,9.357,266.644


In [41]:
df4.isna().sum()

population_x                                0
ranking                                     0
world_share                                 0
iso_code                                    0
year                                        0
biofuel_share_elec                          0
biofuel_elec_per_capita                     0
carbon_intensity_elec                      40
coal_share_elec                             0
coal_elec_per_capita                        0
electricity_generation                      0
biofuel_electricity                         0
coal_electricity                            0
fossil_electricity                          0
gas_electricity                             0
hydro_electricity                           0
nuclear_electricity                         0
oil_electricity                             0
other_renewable_electricity                 0
other_renewable_exc_biofuel_electricity     0
renewables_electricity                      0
solar_electricity                 

In [42]:
df4.drop(["iso_code","carbon_intensity_elec"], axis=1, inplace=True)

/home/krujo/Anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [43]:
df5 = df4

In [44]:
df5.info()

<class 'pandas.core.frame.DataFrame'>
Index: 67 entries, China to Malta
Data columns (total 43 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   population_x                             67 non-null     int64  
 1   ranking                                  67 non-null     int64  
 2   world_share                              67 non-null     float64
 3   year                                     67 non-null     int64  
 4   biofuel_share_elec                       67 non-null     float64
 5   biofuel_elec_per_capita                  67 non-null     float64
 6   coal_share_elec                          67 non-null     float64
 7   coal_elec_per_capita                     67 non-null     float64
 8   electricity_generation                   67 non-null     float64
 9   biofuel_electricity                      67 non-null     float64
 10  coal_electricity                         67 non-nu

In [45]:
df5.columns

Index(['population_x', 'ranking', 'world_share', 'year', 'biofuel_share_elec',
       'biofuel_elec_per_capita', 'coal_share_elec', 'coal_elec_per_capita',
       'electricity_generation', 'biofuel_electricity', 'coal_electricity',
       'fossil_electricity', 'gas_electricity', 'hydro_electricity',
       'nuclear_electricity', 'oil_electricity', 'other_renewable_electricity',
       'other_renewable_exc_biofuel_electricity', 'renewables_electricity',
       'solar_electricity', 'wind_electricity', 'fossil_cons_per_capita',
       'fossil_share_elec', 'gas_share_elec', 'gas_elec_per_capita',
       'hydro_share_elec', 'hydro_elec_per_capita', 'low_carbon_share_elec',
       'low_carbon_electricity', 'low_carbon_elec_per_capita',
       'nuclear_share_elec', 'nuclear_elec_per_capita', 'oil_share_elec',
       'oil_elec_per_capita', 'other_renewables_elec_per_capita',
       'other_renewables_share_elec', 'per_capita_electricity',
       'renewables_elec_per_capita', 'renewables_share_e

In [46]:
df5.to_csv('data/energy_pop_2020.csv' , index = True)
df3.to_csv('data/df3.csv', index = True)